In [ ]:
# Module Importations
from scipy.stats import reciprocal
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import keras
import numpy as np
import time

print('keras.__version__=', keras.__version__)

In [ ]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import keras_helpers
from src.models import model_evaluation

In [ ]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [ ]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)
evaluation_set = evaluation_set.drop("profile_id", axis = 1)

In [ ]:
# Create targets dataset (Stator yoke, tooth and winding temperature)

# Training Set
stator_training_data = training_set.drop(["stator_yoke", "stator_tooth", "stator_winding"], axis = 1)

stator_label_data = training_set[["stator_yoke", "stator_tooth", "stator_winding"]].copy()

# Evaluation Set
stator_eval_data = evaluation_set.drop(["stator_yoke", "stator_tooth", "stator_winding"], axis = 1)

stator_eval_label = evaluation_set[["stator_yoke", "stator_tooth", "stator_winding"]].copy()

In [ ]:
# Create train & test arrays

In [ ]:
# Multilayer Perceptron (Target - Stator Temperatures)

In [ ]:
# Model Evaluation